In [1]:
import pandas as pds
import numpy as np
import time
import importlib
import random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
from torchdiffeq import odeint_adjoint as odeint

In [3]:
from torchdiffeq import odeint as dto

In [4]:
import ode_models
importlib.reload(ode_models)

<module 'ode_models' from '/alt/applic/user-maint/zq224/WS/torchdiffeq/examples/ode_models.py'>

In [5]:
import baseline_models as baseline
import training_utils

importlib.reload(training_utils)


<module 'training_utils' from '/alt/applic/user-maint/zq224/WS/torchdiffeq/examples/training_utils.py'>

In [21]:
import GRUD
importlib.reload(GRUD)


<module 'GRUD' from '/alt/applic/user-maint/zq224/WS/torchdiffeq/examples/GRUD.py'>

In [7]:
device = torch.device('cuda:' + str(0) if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [8]:
D_TYPE = torch.float32

## Get Data

In [9]:
dat_dict = training_utils.get_data('data/cprd_sample_5_obs.csv.gz', normalize=False)
dat_folds = training_utils.get_fold(dat_dict, fold=5, seed=666)

In [10]:
t, y, y0, t_mask, y_mask, eids = training_utils.get_batch(dat_folds[0]['train'], batch_size=70)


In [12]:
y.shape

torch.Size([105, 70, 1, 9])

In [ ]:
def get_batch_from_eids(dat_dict, eids):
    t_list = [dat_dict[e]['t'] for e in eids]
    t_max = max([len(x) for x in t_list])
    t_padded = [F.pad(x, (0, t_max - len(x)), "constant", -1.).reshape((-1, 1)) for x in t_list]
    t_tensor = torch.cat(t_padded, dim=1)
    t_mask = t_tensor >= 0

    x_list = [dat_dict[e]['x'] for e in eids]
    x_padded = [F.pad(i, (0, 0, 0, 0, 0, 0, 0, t_max - i.shape[0]), "constant", -1.) for i in x_list]
    x_tensor = torch.cat(x_padded, dim=1)
    x_mask = x_tensor >= 0
    x0_tensor = x_tensor[0, ...]

    return t_tensor, x_tensor, x0_tensor, t_mask, x_mask, eids


In [21]:
eids = list(dat_dict.keys())[:5]

In [31]:
t_list = [dat_dict[e]['t'] for e in eids]
x_list = [dat_dict[e]['x'] for e in eids]

In [25]:
t_vec = torch.cat(t_list)
t_tensor, _ = torch.sort(torch.unique(t_vec))

torch.Size([172])

In [30]:
t_tensor.shape

torch.Size([167])

In [47]:
t_dict = {str(t): i for i, t in enumerate(t_tensor.numpy())}

In [33]:
x_list[0].shape

torch.Size([36, 1, 1, 9])

In [34]:
x_tensor = torch.zeros((t_tensor.shape[0], len(eids), 1, x_list[0].shape[-1]))

In [35]:
x_tensor.shape

torch.Size([167, 5, 1, 9])

In [67]:
for m in range(len(eids)):
    tm = t_list[m].numpy()
    for i in range(len(tm)):
        ti = str(tm[i])
        ind = t_dict[ti]
        print(m, ti, ind)
        x_tensor[ind, m, 0, :] = x_list[m][i]

0 0.0 0
0 1.5863013 25
0 1.7260274 28
0 1.7534246 29
0 2.1945205 31
0 2.2328768 32
0 3.1342466 43
0 3.6547945 51
0 3.671233 52
0 3.9123287 57
0 4.764384 69
0 4.786301 70
0 4.7945204 71
0 4.810959 74
0 4.928767 76
0 4.939726 77
0 5.057534 79
0 5.071233 82
0 5.3013697 89
0 5.6191783 90
0 5.627397 91
0 5.649315 92
0 6.4136987 103
0 6.6136985 108
0 6.6465755 109
0 6.6547947 110
0 6.6657534 111
0 7.4602737 118
0 7.6986303 122
0 8.150685 125
0 8.630137 132
0 9.660274 144
0 10.230137 149
0 10.249315 150
0 10.290411 152
0 10.29863 153
1 0.0 0
1 0.016438356 1
1 0.019178081 2
1 0.073972605 3
1 0.09315068 4
1 0.1890411 6
1 0.32328767 11
1 0.53424656 14
1 0.59178084 15
1 1.3452054 19
1 1.4082192 21
1 1.5013698 23
1 1.5890411 26
1 2.6136987 37
1 3.1342466 43
1 4.1260276 61
1 4.131507 62
1 5.1232877 83
1 5.178082 84
1 5.2246575 86
2 0.0 0
2 0.22739726 8
2 1.0493151 16
2 2.5945206 36
2 3.2191782 45
2 3.2986302 47
2 3.509589 50
2 3.7369864 53
2 3.9671233 59
2 4.367123 65
2 4.5671234 68
2 4.7972603 72


In [64]:
t_list[0]

tensor([ 0.0000,  1.5863,  1.7260,  1.7534,  2.1945,  2.2329,  3.1342,  3.6548,
         3.6712,  3.9123,  4.7644,  4.7863,  4.7945,  4.8110,  4.9288,  4.9397,
         5.0575,  5.0712,  5.3014,  5.6192,  5.6274,  5.6493,  6.4137,  6.6137,
         6.6466,  6.6548,  6.6658,  7.4603,  7.6986,  8.1507,  8.6301,  9.6603,
        10.2301, 10.2493, 10.2904, 10.2986])

In [72]:
t_tensor[166]

tensor(11.6274)

In [73]:
x_tensor[166, -1, 0, :]

tensor([-0.5872,     nan, -1.7097,     nan,     nan,     nan, -1.1908,     nan,
            nan])

In [74]:
x_list[-1][-1, ...]

tensor([[[-0.5872,     nan, -1.7097,     nan,     nan,     nan, -1.1908,
              nan,     nan]]])

In [76]:
x_nan = torch.isnan(x_tensor)
x_mask =  ~x_nan
x_tensor[x_nan] = 0.

In [11]:
def get_batch_from_eids_stack_t(dat_dict, eids):
    t_list = [dat_dict[e]['t'] for e in eids]
    x_list = [dat_dict[e]['x'] for e in eids]
    t_vec = torch.cat(t_list)
    t_tensor, _ = torch.sort(torch.unique(t_vec))
    t_dict = {str(t): i for i, t in enumerate(t_tensor.numpy())}
    x_tensor = torch.zeros((t_tensor.shape[0], len(eids), 1, x_list[0].shape[-1]))
    x_mask = torch.zeros_like(x_tensor, dtype=torch.bool)
    for m in range(len(eids)):
        tm = t_list[m].numpy()
        for i in range(len(tm)):
            ti = str(tm[i])
            ind = t_dict[ti]
            x_tensor[ind, m, 0, :] = x_list[m][i]
            x_mask[ind, m, 0, :] = ~torch.isnan(x_list[m][i])
    x_tensor[torch.isnan(x_tensor)] = 0.
    x0_tensor = x_tensor[0, ...]
    
    return t_tensor, x_tensor, x0_tensor, None, x_mask, eids

def get_batch_stack_t(dat_dict, batch_size, seed=42):
    random.seed(seed)
    eids = random.sample(list(dat_dict.keys()), batch_size)
    return get_batch_from_eids_stack_t(dat_dict, eids)


In [12]:
t, y, y0, t_mask, y_mask, eids = get_batch_stack_t(dat_dict, batch_size=500)

In [108]:
y.device

device(type='cpu')

In [109]:
len(t)

3883

In [100]:
torch.sum(y_mask)

tensor(5391)

In [101]:
y_mask.shape

torch.Size([1688, 100, 1, 9])

In [102]:
1688 * 100 * 9 - torch.sum(y_mask)

tensor(1513809)

In [103]:
torch.sum(y != 0)

tensor(5391)

In [87]:
y_mask[:, 0, 0, 0]

tensor([False,  True,  True,  ...,  True,  True,  True])

In [91]:
y_mask.dtype

torch.bool

In [95]:
xb_mask = torch.zeros_like(y, dtype=torch.bool)

In [96]:
xb_mask[:, 0, 0, 0]

tensor([False, False, False,  ..., False, False, False])

In [89]:
torch.zeros((3,2,1))

tensor([[[0.],
         [0.]],

        [[0.],
         [0.]],

        [[0.],
         [0.]]])

In [227]:
y.shape

torch.Size([18, 2, 1, 9])

In [ ]:
def get_input_for_grud():
    

In [149]:
# Training one step

# parameters
batch_size = 6
encode_t = 1.
dim_y = 9
dim_hidden = 50
dim_decoder_mlp_hidden = 20

# models
gru = GRUD.GRUD(input_size=dim_y, hidden_size=50, output_last=True)
lstm_decoder = baseline_models.BaselineTimeLSTM(dim_hidden, dim_hidden, dim_hidden)
decoder = Decoder(dim_hidden, dim_y, dim_decoder_mlp_hidden)


# data

t, y, y0, t_mask, y_mask, eids = get_batch_stack_t(dat_dict, batch_size=batch_size)
enc_step = int(torch.sum(t<encode_t))

t_delta = t[1:] - t[:-1]
t_delta = torch.cat((torch.zeros((1,)), t_delta))
t_delta_mat = t_delta.reshape((-1, 1, 1, 1)).repeat((1, y.shape[1], 1, y.shape[-1]))
last_y_mat = torch.cat((y[0:1, ...], y[:-1, ...]), dim=0)

for i in range(1, t_delta_mat.shape[0]):
    last_delta = t_delta_mat[i-1, ...]
    last_mask = y_mask[i-1, ...].to(last_delta)
    t_delta_mat[i, ...] = t_delta_mat[i, ...] + last_delta * (1 - last_mask)
    last_y_mat[i, ...] = last_y_mat[i, ...] + last_y_mat[i-1, ...] * (1 - last_mask)

y_target = y[enc_step:, ...].squeeze().to(device) # y is needed for loss
y_target_mask = y_mask[enc_step:, ...].squeeze().to(device)
gru_d_input = torch.cat((y, last_y_mat, y_mask.to(y), t_delta_mat), dim=2)[:enc_step, ...].permute((1, 2, 0, 3)).to(device)

res = gru(gru_d_input)

# tmat at decode
t_delta_dec_mat = t_delta[enc_step:].reshape((-1, 1)).repeat((1, y.shape[1])).to(device)

y_pred = lstm_decoder(res, t_delta_dec_mat)
y_out = decoder(y_pred)

y_out.shape

loss = torch.mean(torch.abs(y_out[y_target_mask] - y_target[y_target_mask]))

In [150]:
loss.item()

0.7097055912017822

In [145]:
y.shape

torch.Size([68, 6, 1, 9])

In [127]:
t[:int(torch.sum(t<1))]

tensor([0.0000, 0.9233, 0.9808])

In [13]:
t, y, y0, t_mask, y_mask, eids = get_batch_stack_t(dat_dict, batch_size=2)

In [14]:
t_delta = t[1:] - t[:-1]
t_delta = torch.cat((torch.zeros((1,)), t_delta))
t_delta_mat = t_delta.reshape((-1, 1, 1, 1)).repeat((1, y.shape[1], 1, y.shape[-1]))

last_y_mat = torch.cat((y[0:1, ...], y[:-1, ...]), dim=0)

for i in range(1, t_delta_mat.shape[0]):
    last_delta = t_delta_mat[i-1, ...]
    last_mask = y_mask[i-1, ...].to(last_delta)
    t_delta_mat[i, ...] = t_delta_mat[i, ...] + last_delta * (1 - last_mask)
    last_y_mat[i, ...] = last_y_mat[i, ...] + last_y_mat[i-1, ...] * (1 - last_mask)

gru_d_input = torch.cat((y, last_y_mat, y_mask.to(y), t_delta_mat), dim=2).permute((1, 2, 0, 3))

gru_d_input = gru_d_input.to(device)

In [15]:
gru_d_input.device

device(type='cuda', index=0)

In [138]:
importlib.reload(GRUD)

<module 'GRUD' from '/alt/applic/user-maint/zq224/WS/torchdiffeq/examples/GRUD.py'>

In [88]:
x_mean = torch.zeros_like(y).squeeze().permute((1, 0, 2))

gru = GRUD.GRUD(input_size=9, cell_size=None, hidden_size=50, X_mean=x_mean, output_last=True)

In [89]:
x_mean.shape

torch.Size([2, 18, 9])

In [90]:
res = gru(gru_d_input)

In [91]:
res.shape

torch.Size([2, 50])

In [92]:
y0.shape

torch.Size([2, 1, 9])

In [104]:
import baseline_models
importlib.reload(baseline_models)


<module 'baseline_models' from '/alt/applic/user-maint/zq224/WS/torchdiffeq/examples/baseline_models.py'>

In [105]:
lstm_decoder = baseline_models.BaselineTimeLSTM(50, 50, 50)


In [106]:
t_mat = t_delta.reshape((-1, 1)).repeat((1, y.shape[1])).to(device)


In [107]:
t_mat.device

device(type='cuda', index=0)

In [108]:
y_pred = lstm_decoder(res, t_mat)

In [109]:
y_pred.shape

torch.Size([18, 2, 50])

In [114]:
DEVICE = torch.device('cuda:' + str(0) if torch.cuda.is_available() else 'cpu')

class Decoder(nn.Module):

    def __init__(self, latent_dim=4, obs_dim=2, nhidden=20, device=DEVICE):
        super(Decoder, self).__init__()
        self.relu = nn.ReLU(inplace=True).to(device)
        self.fc1 = nn.Linear(latent_dim, nhidden).to(device)
        self.fc2 = nn.Linear(nhidden, obs_dim).to(device)

    def forward(self, z):
        out = self.fc1(z)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [115]:
decoder = Decoder(50, 9, 20)

In [116]:
y_out = decoder(y_pred)

In [117]:
y_out.shape

torch.Size([18, 2, 9])

In [286]:
device

device(type='cuda', index=1)

In [258]:
last_y_mat[:, 0, 0, 6]

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.2255, 0.2255, 0.3304, 0.3304, 0.3304,
        0.3304, 0.3304, 0.3304, 0.3304, 0.3304, 0.3304, 0.3304, 0.3304, 0.3304])

In [255]:
y[:, 0, 0, 6]

tensor([0.0000, 0.0000, 0.0000, 0.2255, 0.0000, 0.3304, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000])

In [256]:
y_mask[:, 0, 0, 6]

tensor([False, False, False,  True, False,  True, False, False, False, False,
        False, False, False, False, False, False, False, False])

In [223]:
y.shape

torch.Size([18, 2, 1, 9])

In [222]:
y_mask[:, 1, 0, :]

tensor([[False,  True, False, False, False, False, False, False, False],
        [False, False,  True, False, False, False,  True, False, False],
        [False, False,  True, False, False, False,  True, False, False],
        [False, False, False, False, False, False, False, False, False],
        [False, False,  True, False, False, False,  True, False, False],
        [False, False, False, False, False, False, False, False, False],
        [False, False,  True, False, False, False,  True, False, False],
        [False, False,  True, False, False, False,  True, False, False],
        [False, False, False, False, False, False, False, False, False],
        [False, False,  True, False, False, False,  True, False, False],
        [False, False, False, False, False, False, False, False, False],
        [False, False,  True, False, False, False,  True, False, False],
        [False, False,  True, False, False, False,  True, False, False],
        [False,  True, False,  True,  True, False, 

In [162]:
t_mat = t.reshape((-1, 1)).repeat((1, 500))

In [163]:
t_mat.shape

torch.Size([3883, 500])

In [159]:
t_vec = t.reshape((-1, 1))
t_vec.shape

torch.Size([3883, 1])

In [160]:
t_mat = t_vec.repeat((1, 500))

In [161]:
t_mat.shape

torch.Size([3883, 500])

# Training

## Baseline - Average

In [48]:
t, y, y0, t_mask, y_mask, eids = get_batch(dat_dict, batch_size=7000)
torch.mean(torch.abs(y))

tensor(0.9626)

In [38]:
y0.shape

torch.Size([70, 1, 4])

## Baseline - LSTM

### LSTM with time info

In [24]:
niters = 5000
test_freq = 100
batch_size = 500
input_dim = output_dim = 4
n_hidden = 50
model_path = 'models/cprd_lstm.pth'

base_time_lstm = baseline.BaselineTimeLSTM(input_dim, n_hidden, output_dim)

optimizer = optim.Adam(base_time_lstm.parameters(), lr=1e-3)


In [25]:
def base_time_lstm_loss_func(t, y, y0, t_mask, y_mask, eids):
    y_pred = base_time_lstm(y0, t)
    loss = torch.mean(torch.abs(y_pred[y_mask.squeeze()] - y[y_mask]))
    return loss

def base_time_lstm_save_func():
    model_path = 'models/cprd_lstm.pth'
    torch.save(base_time_lstm.state_dict(), model_path)

In [99]:
training_utils.training_loop(niters, 
                    dat_folds[0], 
                    batch_size, 
                    optimizer, 
                    test_freq, 
                    base_time_lstm_loss_func, 
                    base_time_lstm_save_func)

Iter 0100 | Total Loss 0.525657
Iter 0200 | Total Loss 0.447242
Iter 0300 | Total Loss 0.412594
Iter 0400 | Total Loss 0.391881
Iter 0500 | Total Loss 0.384577
Iter 0600 | Total Loss 0.378533
Iter 0700 | Total Loss 0.374993
Iter 0800 | Total Loss 0.372451
Iter 0900 | Total Loss 0.370759
Iter 1000 | Total Loss 0.369180
Iter 1100 | Total Loss 0.368492
Iter 1200 | Total Loss 0.368420
Iter 1300 | Total Loss 0.367803
Iter 1400 | Total Loss 0.367126
Iter 1500 | Total Loss 0.367088
Iter 1600 | Total Loss 0.366410
Iter 1700 | Total Loss 0.366708
Iter 1800 | Total Loss 0.366041
Iter 1900 | Total Loss 0.365523
Iter 2000 | Total Loss 0.365653
Iter 2100 | Total Loss 0.365256
Iter 2200 | Total Loss 0.364898
Iter 2300 | Total Loss 0.365341
Iter 2400 | Total Loss 0.364773
Iter 2500 | Total Loss 0.364827
Iter 2600 | Total Loss 0.364687
Iter 2700 | Total Loss 0.363944
Iter 2800 | Total Loss 0.363720
Iter 2900 | Total Loss 0.364264
Iter 3000 | Total Loss 0.363748
Iter 3100 | Total Loss 0.363850
Iter 320

(tensor(0.3622), 267.44750809669495)

In [26]:
base_time_lstm = baseline.BaselineTimeLSTM(input_dim, n_hidden, output_dim)
base_time_lstm.load_state_dict(torch.load('models/cprd_lstm.pth'))
base_time_lstm.eval()

BaselineTimeLSTM(
  (lstm): LSTM(5, 50)
  (lin): Linear(in_features=50, out_features=4, bias=True)
)

In [27]:
lstm_loss = base_time_lstm_loss_func(*training_utils.get_all(dat_folds[0]['test']))

In [28]:
lstm_loss

tensor(0.3622, grad_fn=<MeanBackward0>)

## latent ODE

In [130]:
# run backward lstm to infer initial condition

class EncoderLSTM(nn.Module):

    def __init__(self, input_dim, hidden_dim, output_dim):
        super(EncoderLSTM, self).__init__()
        self.hidden_dim = hidden_dim

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(input_dim + 1, hidden_dim)

        # The linear layer that maps from hidden state space to output space
        self.lin = nn.Linear(hidden_dim, output_dim)

    def forward(self, y, t):
        # y and t are the first k observations
        
        batch_size = y.shape[1]
        dim_y = y.shape[-1]
        t_max = t.shape[0]
        
        t = t.reshape((-1, 1)).repeat((t_max, batch_size, 1))
        y = y.view((t_max, batch_size, dim_y))
        
        y_in = torch.cat((y, t), dim=-1)

        hidden = None
        
        for t in reversed(range(t_max)):
            obs = y_in[t:t+1, ...]
            out, hidden = self.lstm(obs, hidden)
        out_linear = self.lin(out)
        
        return out_linear.permute((1, 0, 2))


In [126]:
a = torch.tensor([1,2,3])

In [129]:
a.reshape((-1, 1)).repeat((2,3))

tensor([[1, 1, 1],
        [2, 2, 2],
        [3, 3, 3],
        [1, 1, 1],
        [2, 2, 2],
        [3, 3, 3]])

In [131]:
class Decoder(nn.Module):

    def __init__(self, latent_dim=4, obs_dim=2, nhidden=20):
        super(Decoder, self).__init__()
        self.relu = nn.ReLU(inplace=True)
        self.fc1 = nn.Linear(latent_dim, nhidden)
        self.fc2 = nn.Linear(nhidden, obs_dim)

    def forward(self, z):
        out = self.fc1(z)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [132]:
niters = 100
batch_size = 100
step_size = 1./12
test_freq = 10

In [133]:
encoder = EncoderLSTM(9, 50, 10)
latent_ode = ode_models.ODEFunc0(dim_y=10)
decoder = Decoder(10, 9, 20)

optimizer = optim.Adam(list(encoder.parameters()) + list(latent_ode.parameters()) + list(decoder.parameters()), lr=1e-3)


In [134]:
def enc_dec_ode_loss_func(t, y, y0, t_mask, y_mask, eids):
    init_cond = encoder(y[:1, ...], t[:1])
    # y_mask[:2, ...] = False
    latent_y = odeint(latent_ode, init_cond, t)
    pred_y = decoder(latent_y)
    loss = torch.mean(torch.abs(pred_y[y_mask] - y[y_mask]))
    return loss

def enc_dec_ode_save_func():
    model_path = 'exp2-models/enc-dec-{}.pth'
    torch.save(encoder.state_dict(), model_path.format('encoder'))
    torch.save(decoder.state_dict(), model_path.format('decoder'))
    torch.save(latent_ode.state_dict(), model_path.format('ode'))

In [141]:
training_utils.training_loop(niters, 
                    dat_folds[0], 
                    batch_size, 
                    optimizer, 
                    test_freq, 
                    enc_dec_ode_loss_func, 
                    enc_dec_ode_save_func,
                    'stack')

Iter 0010 | Total Loss 0.758063
Iter 0020 | Total Loss 0.754229
Iter 0030 | Total Loss 0.751824
Iter 0040 | Total Loss 0.751656
Iter 0050 | Total Loss 0.749535
Iter 0060 | Total Loss 0.750715
Iter 0070 | Total Loss 0.747994
Iter 0080 | Total Loss 0.746622
Iter 0090 | Total Loss 0.744011
Iter 0100 | Total Loss 0.742634


(tensor(0.7426), 1591.252745628357)

In [218]:

model_path = 'models/enc-dec-{}.pth'

encoder = EncoderLSTM(4, 50, 10)
encoder.load_state_dict(torch.load(model_path.format('encoder')))
encoder.eval()

latent_ode = ode_models.ODEFunc0(dim_y=10)
latent_ode.load_state_dict(torch.load(model_path.format('ode')))
latent_ode.eval()

decoder = Decoder(10, 4, 20)
decoder.load_state_dict(torch.load(model_path.format('decoder')))
decoder.eval()

Decoder(
  (relu): ReLU(inplace=True)
  (fc1): Linear(in_features=10, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=4, bias=True)
)

In [219]:
latent_ode_loss = enc_dec_ode_loss_func(*training_utils.get_all(dat_folds[0]['test']))
latent_ode_loss

tensor(0.3643, grad_fn=<MeanBackward0>)

## FS Latent ODE

In [203]:
niters = 2000
batch_size = 500
step_size = 1./12
test_freq = 50

In [204]:
encoder = EncoderLSTM(4, 50, 10)
fs_ode = ode_models.FSODE(input_dim=10, hidden_dim=50)
decoder = Decoder(10, 4, 20)

optimizer = optim.Adam(list(encoder.parameters()) + list(fs_ode.parameters()) + list(decoder.parameters()), lr=1e-3)


In [205]:
def enc_dec_fs_ode_loss_func(t, y, y0, t_mask, y_mask, eids):
    init_cond = encoder(y[:1, ...], t[:1])
    # y_mask[:2, ...] = False
    latent_y = dto(fs_ode, init_cond, t, method='euler_par', options={'step_size': step_size})
    pred_y = decoder(latent_y)
    loss = torch.mean(torch.abs(pred_y[y_mask] - y[y_mask]))
    return loss

def enc_dec_fs_ode_save_func():
    model_path = 'models/enc-dec-fs-{}.pth'
    torch.save(encoder.state_dict(), model_path.format('encoder'))
    torch.save(decoder.state_dict(), model_path.format('decoder'))
    torch.save(fs_ode.state_dict(), model_path.format('ode'))

In [207]:
training_utils.training_loop(niters, 
                    dat_folds[0], 
                    batch_size, 
                    optimizer, 
                    test_freq, 
                    enc_dec_fs_ode_loss_func, 
                    enc_dec_fs_ode_save_func)

Iter 0050 | Total Loss 0.368517
Iter 0100 | Total Loss 0.368873
Iter 0150 | Total Loss 0.368728
Iter 0200 | Total Loss 0.368591
Iter 0250 | Total Loss 0.367557
Iter 0300 | Total Loss 0.369183
Iter 0350 | Total Loss 0.368086
Iter 0400 | Total Loss 0.367389
Iter 0450 | Total Loss 0.367729
Iter 0500 | Total Loss 0.366998
Iter 0550 | Total Loss 0.367249
Iter 0600 | Total Loss 0.366565
Iter 0650 | Total Loss 0.366378
Iter 0700 | Total Loss 0.366370
Iter 0750 | Total Loss 0.365667
Iter 0800 | Total Loss 0.365757
Iter 0850 | Total Loss 0.365701
Iter 0900 | Total Loss 0.365846
Iter 0950 | Total Loss 0.365699
Iter 1000 | Total Loss 0.365524


(tensor(0.3655), 1887.9040415287018)

In [209]:
fs_ode_loss = enc_dec_fs_ode_loss_func(*training_utils.get_all(dat_folds[0]['test']))
fs_ode_loss

tensor(0.3654, grad_fn=<MeanBackward0>)

## HO Latent ODE

In [222]:
niters = 2000
batch_size = 500
step_size = 1./12
test_freq = 50

In [225]:
encoder = EncoderLSTM(4, 50, 10)
ho_ode = ode_models.HigherOrderOdeNoInit(dim=5, order=2, hidden_size=50)
decoder = Decoder(10, 4, 20)

optimizer = optim.Adam(list(encoder.parameters()) + list(ho_ode.parameters()) + list(decoder.parameters()), lr=1e-3)


In [226]:
def enc_dec_ho_ode_loss_func(t, y, y0, t_mask, y_mask, eids):
    init_cond = encoder(y[:1, ...], t[:1])
    # y_mask[:2, ...] = False
    latent_y = dto(ho_ode, init_cond, t, method='euler_par', options={'step_size': step_size})
    pred_y = decoder(latent_y)
    loss = torch.mean(torch.abs(pred_y[y_mask] - y[y_mask]))
    return loss

def enc_dec_ho_ode_save_func():
    model_path = 'models/enc-dec-ho-{}.pth'
    torch.save(encoder.state_dict(), model_path.format('encoder'))
    torch.save(decoder.state_dict(), model_path.format('decoder'))
    torch.save(ho_ode.state_dict(), model_path.format('ode'))

In [227]:
training_utils.training_loop(niters, 
                    dat_folds[0], 
                    batch_size, 
                    optimizer, 
                    test_freq, 
                    enc_dec_ho_ode_loss_func, 
                    enc_dec_ho_ode_save_func)

Iter 0050 | Total Loss 0.600317
Iter 0100 | Total Loss 0.549817
Iter 0150 | Total Loss 0.518479
Iter 0200 | Total Loss 0.498119
Iter 0250 | Total Loss 0.479272
Iter 0300 | Total Loss 0.458242
Iter 0350 | Total Loss 0.435501
Iter 0400 | Total Loss 0.420988
Iter 0450 | Total Loss 0.410365
Iter 0500 | Total Loss 0.398915
Iter 0550 | Total Loss 0.386060
Iter 0600 | Total Loss 0.381689
Iter 0650 | Total Loss 0.378942
Iter 0700 | Total Loss 0.377669
Iter 0750 | Total Loss 0.375717
Iter 0800 | Total Loss 0.375116
Iter 0850 | Total Loss 0.374617
Iter 0900 | Total Loss 0.373847
Iter 0950 | Total Loss 0.372793
Iter 1000 | Total Loss 0.373130
Iter 1050 | Total Loss 0.371686
Iter 1100 | Total Loss 0.372228
Iter 1150 | Total Loss 0.371427
Iter 1200 | Total Loss 0.371782
Iter 1250 | Total Loss 0.370440
Iter 1300 | Total Loss 0.370212
Iter 1350 | Total Loss 0.370274
Iter 1400 | Total Loss 0.369955
Iter 1450 | Total Loss 0.369631
Iter 1500 | Total Loss 0.369783
Iter 1550 | Total Loss 0.368594
Iter 160

(tensor(0.3672), 2827.6357975006104)

In [187]:
y[:2, ...].shape

torch.Size([2, 11, 1, 4])

In [186]:
t[:2].shape

torch.Size([2, 11])

In [183]:
init_cond = encoder(y[:1, ...], t[:1])

RuntimeError: invalid argument 0: Tensors must have same number of dimensions: got 2 and 3 at /opt/conda/conda-bld/pytorch_1570910687650/work/aten/src/TH/generic/THTensor.cpp:680

In [171]:

t, y, y0, t_mask, y_mask, eids = training_utils.get_batch(dat_folds[0]['train'], batch_size=11)
optimizer.zero_grad()
init_cond = encoder(y[:2, ...], t[:2])
# y_mask[:2, ...] = False
latent_y = dto(latent_ode, init_cond, t, method='euler_par', options={'step_size': step_size})
pred_y = decoder(latent_y)
loss = torch.mean(torch.abs(pred_y[y_mask] - y[y_mask]))
loss.backward()
optimizer.step()

In [167]:
out_y.shape

torch.Size([15, 11, 1, 4])

In [168]:
pred_y.shape

torch.Size([15, 11, 1, 10])

In [158]:
t.shape

torch.Size([15, 11])

In [146]:
for i in range(1000):
    t, y, y0, t_mask, y_mask, eids = training_utils.get_batch(dat_folds[0]['train'], batch_size=11)
    opt.zero_grad()
    init_cond = encoder(y[:2, ...], t[:2])
    
    pred_y = dto(latent_ode, init_cond, t, method='euler_par', options={'step_size': step_size})
    
    
    
    loss = torch.mean(torch.abs(init_cond.squeeze() - y0.squeeze()))
    loss.backward()
    opt.step()

In [147]:
loss.item()

0.006156797520816326

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [1.8000, 1.2438, 2.1178, 0.8356, 3.2137, 0.6877, 0.9836, 3.9123, 1.0329,
         2.1178, 1.0164],
        [3.0247, 1.9425, 2.9260, 2.8493, 4.3616, 3.2630, 2.0904, 8.8082, 1.4384,
         2.6301, 2.0795]])

In [152]:
init_cond.permute((1, 0, 2)).shape

torch.Size([11, 1, 4])

In [149]:
y0.shape

torch.Size([11, 1, 4])

In [95]:
torch.mean(torch.abs(func.init_cond_mat[:, :, 4:]))

tensor(0.7702)

In [96]:
torch.mean(torch.abs(func.init_cond_mat[:, :, :4]))

tensor(0.7975)

In [107]:
with torch.no_grad():
    t, y, y0, t_mask, y_mask, eids = get_all(dat_folds[0]['val'])
    func.set_init_cond(eids)
    init_zeros = torch.zeros_like(func.init_cond)

    pred_y = dto(func, init_zeros, t, method='euler_par', options={'step_size': step_size})
    pred_y_final = (pred_y  + func.init_cond)[..., :func.dim]

    loss = torch.mean(torch.abs(pred_y_final[y_mask] - y[y_mask]))


In [112]:
func.init_cond[:5,0, ]

tensor([[ 0.0748,  0.1314,  0.0741, -0.2251,  0.2084,  0.0000,  0.5503,  0.0850],
        [-1.0692,  1.0429,  0.0741, -1.0025,  0.4438, -0.1469,  0.4507, -0.0696],
        [-0.3912, -0.1421,  0.0182, -0.6570,  0.1905,  0.0000,  0.2935, -0.1618],
        [-0.1370,  0.1314,  0.3538,  1.4160,  0.0709, -0.2287,  0.3508, -0.5056],
        [ 0.6680, -1.3271, -2.1075,  0.8114, -0.8754, -0.4185,  2.8251, -0.9914]])

In [111]:
y0[:5,0, :4]

tensor([[ 0.0748,  0.1314,  0.0741, -0.2251],
        [-1.0692,  1.0429,  0.0741, -1.0025],
        [-0.3912, -0.1421,  0.0182, -0.6570],
        [-0.1370,  0.1314,  0.3538,  1.4160],
        [ 0.6680, -1.3271, -2.1075,  0.8114]])

In [114]:
pred_y_final[:5, 0, 0, :]

tensor([[ 0.0748,  0.1314,  0.0741, -0.2251],
        [ 0.2966,  0.3205,  0.4992,  0.1014],
        [ 0.3616,  0.3970,  0.5153,  0.2271],
        [ 0.3939,  0.4275,  0.5174,  0.2844],
        [ 0.4475,  0.4661,  0.5184,  0.3721]])

In [115]:
y[:5, 0, 0, :]

tensor([[ 0.0748,  0.1314,  0.0741, -0.2251],
        [ 0.2867,  0.1314,  0.6335, -0.1387],
        [ 0.3291,  0.1314, -0.1496,  0.0340],
        [ 0.4985, -0.2332, -0.4853, -1.0889],
        [-0.0523,  0.1314,  0.0741, -0.3115]])

In [198]:
ii = 0

start = time.time()
for itr in range(1, niters + 1):
    
    t, y, y0, t_mask, y_mask, eids = get_batch(dat_dict, batch_size, itr+500)

    optimizer.zero_grad()
    
    func.set_init_cond(eids)
    init_zeros = torch.zeros_like(func.init_cond)
    
    pred_y = dto(func, init_zeros, t, method='euler_par', options={'step_size': step_size})
    pred_y_final = (pred_y  + func.init_cond)[..., :func.dim]
    
    loss = torch.mean(torch.abs(pred_y_final[y_mask] - y[y_mask]))
    loss.backward()
    optimizer.step()

    if itr % test_freq == 0:
        with torch.no_grad():
            print('Iter {:04d} | Total Loss {:.6f}'.format(itr, loss.item()))
            ii += 1

end = time.time()


Iter 0050 | Total Loss 0.426231
Iter 0100 | Total Loss 0.597094
Iter 0150 | Total Loss 0.583271
Iter 0200 | Total Loss 1.400030
Iter 0250 | Total Loss 0.738257
Iter 0300 | Total Loss 0.420469
Iter 0350 | Total Loss 0.611337
Iter 0400 | Total Loss 0.443519
Iter 0450 | Total Loss 0.491005
Iter 0500 | Total Loss 0.892621
Iter 0550 | Total Loss 0.768308
Iter 0600 | Total Loss 0.525871
Iter 0650 | Total Loss 0.453953
Iter 0700 | Total Loss 1.970526
Iter 0750 | Total Loss 0.988136
Iter 0800 | Total Loss 2.543571
Iter 0850 | Total Loss 1.738551
Iter 0900 | Total Loss 0.450546
Iter 0950 | Total Loss 0.405119
Iter 1000 | Total Loss 0.566877
Iter 1050 | Total Loss 0.449112
Iter 1100 | Total Loss 0.478447
Iter 1150 | Total Loss 0.420794
Iter 1200 | Total Loss 0.375969
Iter 1250 | Total Loss 0.641680
Iter 1300 | Total Loss 0.411319
Iter 1350 | Total Loss 0.385653
Iter 1400 | Total Loss 0.409560
Iter 1450 | Total Loss 0.422992
Iter 1500 | Total Loss 0.390408


In [201]:
model_path = 'models/cprd_ho2.pth'
torch.save(func.state_dict(), model_path)


In [202]:
# func1 = ode_models.HigherOrderOde(dat_dict,batch_size=batch_size, dim=4, order=2, hidden_size=50)
# func1.load_state_dict(torch.load(model_path))
# func1.eval()
# func1.init_cond_mat.requires_grad = False


In [199]:
(end-start)/60

41.17963133653005

In [182]:
itr=498

In [183]:
t, y, y0, t_mask, y_mask, eids = get_batch(dat_dict, batch_size, itr)

In [184]:
eids[0]

'8423206'

In [158]:
dat_dict['14260073']

{'t': tensor([0.0000, 3.1205, 4.7178]),
 'x': tensor([[[[0.4524, 2.7864, 2.9481, 1.3142]]],
 
 
         [[[0.1569, 1.9626, 1.4286, 0.2870]]],
 
 
         [[[0.7479, 1.6880, 1.6537, 0.8006]]]])}

In [156]:
y0[0, ...]

tensor([[0.4524, 2.7864, 2.9481, 1.3142]])

In [168]:
idx = np.array([func.eid_to_id[x] for x in eids])
id_torch = torch.from_numpy(idx)
#  self.init_cond = self.init_cond_mat[id_torch, ...]
id_torch[0]

tensor(5238)

In [169]:
func.init_cond_mat[5238]

tensor([[ 0.4512,  2.7882,  2.9492,  1.3124, -0.0830, -0.2435, -0.4933, -0.3075]],
       grad_fn=<SelectBackward>)

In [161]:
func.init_cond[0]

tensor([[-0.1238, -0.6072, -1.0318,  0.8855, -0.1671,  0.3526,  0.2405, -0.1994]],
       grad_fn=<SelectBackward>)

In [162]:
func.init_cond_mat

torch.Size([7010, 1, 8])

In [192]:
func_untrain = ode_models.HigherOrderOde(dat_dict,batch_size=batch_size, dim=4, order=2, hidden_size=50)


In [193]:
torch.mean(torch.abs(func.init_cond_mat[..., :4] - func_untrain.init_cond_mat[..., :4]))

tensor(0., grad_fn=<MeanBackward0>)

In [166]:
func.init_cond_mat[:5, 0, :4]

tensor([[-0.1154,  0.6854,  1.9798,  2.4262],
        [-1.1180,  0.0436, -0.2041,  0.8874],
        [ 0.4524, -0.5087, -0.8788, -0.2265],
        [ 2.0107, -0.1345,  0.5280,  0.5414],
        [ 0.1557, -0.3421,  0.3031, -0.1577]], grad_fn=<SliceBackward>)

In [167]:
func_untrain.init_cond_mat[:5, 0, :4]

tensor([[-0.0964,  0.6812,  1.9914,  2.4268],
        [-1.1096,  0.0405, -0.2035,  0.8862],
        [ 0.4524, -0.5087, -0.8788, -0.2265],
        [ 2.0144, -0.1426,  0.5282,  0.5438],
        [ 0.1569, -0.3256,  0.3031, -0.1409]], grad_fn=<SliceBackward>)

In [185]:
func.init_cond_mat[:5, 0, -4:]

tensor([[-0.6105,  0.4214,  2.3506, -0.3816],
        [-0.0939,  0.3392,  0.6464, -0.7820],
        [-1.3570,  0.3796, -0.4085,  0.0888],
        [-0.0635, -0.4829, -0.3766, -0.1169],
        [-0.1704,  0.3768, -0.1794,  0.4243]], grad_fn=<SliceBackward>)

In [186]:
func_untrain.init_cond_mat[:5, 0, -4:]

tensor([[-0.5927,  0.4283,  2.3702, -0.4005],
        [-0.0888,  0.3464,  0.6390, -0.7918],
        [-1.3570,  0.3796, -0.4085,  0.0888],
        [-0.0617, -0.4903, -0.3837, -0.1250],
        [-0.1813,  0.3930, -0.1933,  0.4411]], grad_fn=<SliceBackward>)

# old stuff

In [171]:
n_processed = 0
step_size = 1./12

s = time.time()
for eid, v in dat_dict.items():
    t = v['t']
    x = v['x']

    optimizer.zero_grad()
    
    
    func.set_init_cond([eid])
    init_zeros = torch.zeros_like(func.init_cond)
    
#     pred_y = odeint(func, init_zeros, t)
    pred_y = dto(func, init_zeros, t, method='euler', options={'step_size': step_size})
    
    pred_y_final = pred_y  + func.init_cond
    loss = torch.mean(torch.abs(pred_y_final[..., :func.dim] - x))
    loss.backward()
    optimizer.step()
    
    n_processed += 1
    
    if n_processed % 500 == 0:
        print('Processed:', n_processed)
#     if n_processed > 100:
#         break
e = time.time()

Processed: 500
Processed: 1000
Processed: 1500
Processed: 2000
Processed: 2500
Processed: 3000
Processed: 3500
Processed: 4000
Processed: 4500
Processed: 5000
Processed: 5500
Processed: 6000
Processed: 6500
Processed: 7000


In [172]:
e - s

175.36765384674072

In [133]:
init_zeros.requires_grad

True

In [122]:
n_processed

2

In [69]:
func.init_cond

Parameter containing:
tensor([[[100.0000,  91.0000, 128.0000,   6.4000, -11.1027,  -1.3878,   5.5513,
           -2.6369]]], requires_grad=True)

In [62]:
func.init_cond.shape

torch.Size([1, 1, 8])

In [88]:
5 % 3

2

In [102]:
n_processed

31

In [91]:
pred_y = odeint(func, init_zeros, t)

In [135]:
v = dat_dict['1092']

t = v['t']
x = v['x']

optimizer.zero_grad()
func.set_init_cond(eid, t, x)
init_zeros = torch.zeros_like(func.init_cond)




AttributeError: 'HigherOrderOde' object has no attribute 'set_init_cond'

In [99]:
pred_y = odeint(func, init_zeros, t)
pred_y_final = pred_y + func.init_cond


In [100]:
loss = torch.mean(torch.abs(pred_y_final[..., :func.dim] - x))
loss.backward()


In [101]:
optimizer.step()

In [139]:
x[0, ...].shape

torch.Size([1, 1, 4])

In [144]:
eid_list = list(dat_dict.keys())
eid_to_id = dict(zip(eid_list, range(len(eid_list))))


In [174]:
dat_dict['1092']

{'t': tensor([0.0000, 0.8548, 4.8959]),
 'x': tensor([[[[ 86.0000,  86.0000, 174.0000,   7.6000]]],
 
 
         [[[ 74.0000,  90.0000, 210.0000,   7.2000]]],
 
 
         [[[ 88.0000,  82.0000, 140.0000,   5.6000]]]])}

In [175]:
dat_dict['3259']

{'t': tensor([0.0000, 2.3781, 5.7644]),
 'x': tensor([[[[ 62.0000,  79.0000, 135.0000,   5.8000]]],
 
 
         [[[ 57.0000,  88.0000, 162.0000,   3.6000]]],
 
 
         [[[ 74.0000,  78.0000, 146.0000,   4.8000]]]])}

In [165]:
x.shape

torch.Size([3, 1, 1, 4])